<a href="https://colab.research.google.com/github/AndresPalacio/AugLy/blob/main/generacion_de_texto_en_espa%C3%B1ol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluación y entrenamiento de generadores de texto en español

En este documento se muestra como probar generadores de texto y luego entrenarlos para que generen texto con sintaxis y vocabulario similar al que le demos como referencia.

**Nota:** Los modelos de español aún no son tan buenos como los de inglés, puede comparar los resultados con los que genera [inferkit](https://app.inferkit.com/generate) por ejemplo.


Adaptación de [Colab](https://github.com/somosnlp/nlp-de-cero-a-cien/blob/main/6_transformers_modelado_del_lenguaje/fine_tune_Spanish_GPT_2.ipynb#scrollTo=k9NXN1YVdC0G) por [Maria Grandury](https://github.com/mariagrandury)


# Setup y evaluación manual

## Instalamos paquetes necesarios

In [ ]:
#!pip install -q transformers
!pip install git+https://github.com/huggingface/transformers
!pip install -q datasets

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-wlpuf45_
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-wlpuf45_
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 596 kB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 39.9 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 895 kB 36.1 MB/s 
  Created wheel for transformers: filename=transformers-4.19.0.dev0-py3-none-any.whl size=4041112 sha256=721368e40d23730bdad264275a6eb56c421d2ec2ee8e7b95e4b721c9190aec89
  Stored in directory: /tmp/pip-ephem-wheel-cache-9xu8l7_y/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully unin

## Elegimos un modelo / checkpoint

Hacer que una computadora aprenda a hablar español es mucho trabajo. Hoy en día lo que se suele hacer es utilizar modelos que ya saben hablar el idioma y luego se los especializa en un estilo particular, en vez de arrancar el aprendizaje de la red desde cero. A esta técnica se le llama fine tuning.

En el último año surgieron varios modelos públicos de español lo cuál facilitó en gran medida este trabajo, pueden ver algunos [acá](https://somosnlp.org/recursos/open-source/modelos).

In [ ]:
from transformers import pipeline

def generate(initial_text, max_length = 100):
  generated_text = generator(initial_text, do_sample=True, pad_token_id=50256, max_length=max_length, device=0)[0]["generated_text"]
  return generated_text


## Probamos el modelo

In [ ]:
generator = pipeline('text-generation', model=model_checkpoint, device=0)

Downloading:   0%|          | 0.00/811 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/487M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
generate('Una mañana de otoño')

'Una mañana de otoño se me ocurrió ir por la pequeña calle de la parte vieja de la ciudad universitaria, la plaza del Príncipe Leopoldo, antes de ver la Iglesia de San Ignacio de Loyola, en el centro, la de los Padres Trinitarios. En su puerta había una vieja torre y desde ahí un puente románico que nos cruzaba por un costado. Pero no, era yo uno de esos que no le'

# Fine Tuning

## Creamos el dataset

Para esto utilizaré un TXT que armé con textos de Borges

In [ ]:
!wget --no-check-certificate 'https://gist.githubusercontent.com/mathigatti/93b680349a44df569735182d72d0724b/raw/97864918e897c45a01ed9ee1c1edf4e54b5697ba/okcupid.csv' -O okcupid.csv

--2022-04-26 17:12:51--  https://gist.githubusercontent.com/mathigatti/93b680349a44df569735182d72d0724b/raw/97864918e897c45a01ed9ee1c1edf4e54b5697ba/okcupid.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1557772 (1.5M) [text/plain]
Saving to: ‘okcupid.csv’

okcupid.csv         100%[===================>]   1.49M  --.-KB/s    in 0.05s   

2022-04-26 17:12:52 (27.9 MB/s) - ‘okcupid.csv’ saved [1557772/1557772]



In [ ]:
import pandas as pd
df = pd.read_csv('/content/okcupid.csv')

def format(keywords, text):
  return f"KEYWORDS: {keywords} | DESCRIPTION: {text}"

df["answer"] = df.apply(lambda x : format(x["keywords"], x["answer"]),axis=1)
df.to_csv("okcupid_formatted.csv",index=False,header=True)

## Tokenizamos el dataset

El texto solo existe en nuestras cabezas, en realidad es solo pixels en una pantalla. A veces nos olvidamos de eso pero entrenando redes neuronales uno se ve obligado a recordarlo.

Las imagenes, el texto y los sonidos son abstracciones generadas a partir de cosas mucho mas elementales. Cuando vemos una imagen nuestros ojos reciben millones de fotones en la retina, neuronas del campo visual procesan independientemente esta información sensada, luego otras neuronas procesan los resultados de las neuronas anteriores y luego de suficiente trabajo en equipo se genera una abstracción que reune toda esa información básica, se concluye que vemos una cara, pero todo surgió de sensar fotones de luz.

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
#@title ⬅ Run this cell to create dataset and save the processed dataset
from datasets import load_dataset

dataset_path = '/content/okcupid_formatted.csv' #@param {type:"string"}
dataset = load_dataset('csv', data_files=[dataset_path], split='train')
#dataset = load_dataset('csv', data_files=dataset_path, split='train')

dataset = dataset.map(lambda x: tokenizer(x['answer']),
                      batch_size = 10000,
                      batched=True,
                      writer_batch_size = 10000,
)

def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

block_size = 1024

dataset = dataset.remove_columns(['attention_mask', 'keywords', "answer"])
#dataset = dataset.map(group_texts, batched=True)

dataset.cleanup_cache_files()
path_ts_dataset = "processed_dataset" #@param {type:"string"}
dataset.save_to_disk(path_ts_dataset)

Using custom data configuration default-2767f61ea124b315
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-2767f61ea124b315/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?ba/s]

## Entrenamos la red

In [ ]:
import os

from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer

from transformers import default_data_collator
data_collator = default_data_collator

def start_trainer(name, model_checkpoint, dataset, epochs=1):
  model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
  model.resize_token_embeddings(len(tokenizer))
  training_args = TrainingArguments(
      output_dir=os.path.join("./", name), #The output directory
      num_train_epochs=epochs, # number of training epochs
      per_device_train_batch_size = 2,
      per_device_eval_batch_size = 2,
      evaluation_strategy = "epoch",
      learning_rate=2e-5,
      weight_decay=0.01,
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      data_collator=data_collator,
      train_dataset=dataset,
      eval_dataset=dataset,
      tokenizer=tokenizer,
  )

  return trainer

trainer = start_trainer("okcupidbot", model_checkpoint, dataset, epochs=1)

loading configuration file https://huggingface.co/flax-community/gpt-2-spanish/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ee7c2d6fbb27a3edcc3f91415462108e3f3c3531f29f0d91960303619ad92e66.75954be0f29e94c10a36ae33108e061f85d7344018ac04f1e66fb38613570c7a
Model config GPT2Config {
  "_name_or_path": "flax-community/gpt-2-spanish",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.0,
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.0,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "s

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6566
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 3283


ValueError: ignored

In [ ]:
trainer.save_model()

Saving model checkpoint to ./okcupidbot
Configuration saved in ./okcupidbot/config.json
Model weights saved in ./okcupidbot/pytorch_model.bin
tokenizer config file saved in ./okcupidbot/tokenizer_config.json
Special tokens file saved in ./okcupidbot/special_tokens_map.json


## Evaluamos los resultados

In [ ]:
from transformers import pipeline
generator = pipeline('text-generation',model='okcupidbot', tokenizer=model_checkpoint)

loading configuration file okcupidbot/config.json
Model config GPT2Config {
  "_name_or_path": "okcupidbot",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.0,
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.0,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.19.0.dev0",
  "use_cache": tru

In [ ]:
generate('KEYWORDS: cara de culo, feliz | DESCRIPTION: ', max_length = 200)

'KEYWORDS: cara de culo, feliz | DESCRIPTION: de cara de culo, feliz | DESCRIPTION: por ser modelo, tengo cara de culo, sexy | DESCRIPTION: por ser modelo | DESCRIPTION: por ser modelo, tener cara de culoEl Gobierno bonaerense hizo entrega formal del lote de un vehículo del programa de construcción del Hospital municipal de La Boca. La delegación llegó al barrio, en el microestadio de la avenida Rivadavia, de manera formal y junto al director y el secretario de Desarrollo Humano, Marcelo Marchi. El jefe comunal expresó en el lugar que “es responsabilidad de la Municipalidad de La Boca, a partir de las medidas tomadas por la gobernadora María Eugenia Vidal, el proyecto de construcción de este hospital, que tendrá una inversión de más d'